In [ ]:
from netCDF4 import Dataset
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import netcdf  
import scipy
import scipy.signal
import matplotlib
#from mpl_toolkits.basemap import Basemap
from scipy.interpolate import Rbf
#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import random
import h5py
from sklearn import preprocessing
import torch.utils.data as Data
import netCDF4 as nc
import seawater as sw
from matplotlib.colors import ListedColormap, Normalize
import cmaps
import matplotlib.ticker as ticker
from scipy.interpolate import interp1d
import scipy.io

In [ ]:

F1=scipy.io.loadmat('G:/ML&C/Response_11.07/Redrawing/Zonal and Meridional Baroclinic Semidiurnal Tidal Currents at MR1 Site.mat')
u_pre=np.array(F1['u_pre'])
v_pre=np.array(F1['v_pre'])
u=np.array(F1['u'])
v=np.array(F1['v'])

F2=scipy.io.loadmat('G:/ML&C/Response_11.07/Redrawing/RMSE.mat')
u_rm=np.array(F2['u_rm'])
v_rm=np.array(F2['v_rm'])
rmseu=np.array(F2['u_incrm'])
rmsev=np.array(F2['v_incrm'])
rmseuc=np.array(F2['u_cohrm'])
rmsevc=np.array(F2['v_cohrm'])

F3=scipy.io.loadmat('G:/ML&C/Easy_redrawing/HKE.mat')
ke_mode1=np.array(F3['ke_mode1'])
ke_mode1_di=np.array(F3['ke_mode1_di'])
ke_mode1_obs=np.array(F3['ke_mode1_obs'])



In [ ]:

#Baroclinic semidiurnal tidal currents

time = np.arange(1320)
time1=np.arange(672)
time2=np.arange(648)
depth =  np.arange(160,1008,8)

fig = plt.figure(figsize=(15, 10), dpi=300)
gs = plt.GridSpec(3, 1)

ax3 = fig.add_subplot(gs[2, 0])
ax1 = fig.add_subplot(gs[0, 0], sharex=ax3)
ax2 = fig.add_subplot(gs[1, 0], sharex=ax3)

f3 = ax3.contourf(time1, depth,(u_pre-u).T, 
               #bias.T, 
                    levels=1000,vmin=-0.15,vmax=0.15,
                    #levels=1000,
               cmap=cmapss, linestyles='none')
f2 = ax2.contourf(time1, depth, u_pre.T,
               #levels=np.arange(-7, -5, 0.05), 
                    levels=1000,  
               cmap=cmapss,linestyles='none')
f1 = ax1.contourf(time1, depth, u.T, 
                    #levels=np.arange(-7,-5,0.05), 
                    levels=1000,
               cmap=cmapss,linestyles='none')


ax3.set_xticks([0,167,335,503,671])
ax3.set_xticklabels(['05/13','05/20','05/27','06/03','06/10'],fontsize=13, fontweight='bold')
ax3.tick_params(axis='x', pad=7) 
ax1.set_yticklabels(np.int64(ax1.get_yticks()), fontsize=13, fontweight='bold')
ax2.set_yticklabels(np.int64(ax2.get_yticks()), fontsize=13, fontweight='bold')
ax3.set_yticklabels(np.int64(ax3.get_yticks()), fontsize=13, fontweight='bold')
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
ax1.set_title('(a) Observed u',fontsize=15, fontweight='bold',loc='left',pad=10)
ax2.set_title('(b) Forecasted u',fontsize=15, fontweight='bold',loc='left',pad=10)
ax3.set_title('(c) Differences',fontsize=15, fontweight='bold',loc='left',pad=10)
ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax1.set_ylabel('Depth (m)',fontsize=15, fontweight='bold')
ax2.set_ylabel('Depth (m)',fontsize=15, fontweight='bold')
ax3.set_ylabel('Depth (m)',fontsize=15, fontweight='bold')
ax3.set_xlabel('Time',fontsize=15, fontweight='bold')

formatter = ticker.FuncFormatter(lambda x, _: f'{x:>5.2f}')
cb0 = plt.colorbar(f1, ax=ax1, shrink=1,pad=0.01,aspect=10, extend='both'
                   #,ticks=ticker.MaxNLocator(nbins=8)
                  )
cb0.ax.set_title('u (m/s)', fontsize=12, pad=10, fontweight='bold')
cb0.set_ticks([-0.15,-0.10,-0.05,0,0.05,0.10,0.15])
cb0.ax.tick_params(which='major', direction='in', length=5, width=1)
cb0.ax.yaxis.set_tick_params(which='major', labelsize=14, width=1, labelcolor='black')
cb0.ax.yaxis.set_major_formatter(formatter)

cb1 = plt.colorbar(f1, ax=ax2, shrink=1,pad=0.01,aspect=10, extend='both')
cb1.ax.set_title('u (m/s)', fontsize=12, pad=10, fontweight='bold')
#cb1.set_width(0.5)
cb1.set_ticks([-0.15,-0.10,-0.05,0,0.05,0.10,0.15])
cb1.ax.tick_params(which='major', direction='in', length=5, width=1)
cb1.ax.yaxis.set_tick_params(which='major', labelsize=14, width=1, labelcolor='black')
cb1.ax.yaxis.set_major_formatter(formatter)

cb2 = plt.colorbar(f1, ax=ax3, shrink=1,pad=0.01,aspect=10, extend='both')
cb2.ax.set_title('u (m/s)', fontsize=12, pad=10, fontweight='bold')
#cb2.set_width(0.5)
cb2.set_ticks([-0.15,-0.10,-0.05,0,0.05,0.10,0.15])
cb2.ax.tick_params(which='major', direction='in', length=5, width=1)
cb2.ax.yaxis.set_tick_params(which='major', labelsize=14, width=1, labelcolor='black')
cb2.ax.yaxis.set_major_formatter(formatter)

ax1.tick_params(axis='x', direction='in', length=6, width=2)
ax1.tick_params(axis='y', direction='in', length=6, width=2)
ax2.tick_params(axis='x', direction='in', length=6, width=2)
ax2.tick_params(axis='y', direction='in', length=6, width=2)
ax3.tick_params(axis='x', direction='in', length=6, width=2)
ax3.tick_params(axis='y', direction='in', length=6, width=2)

plt.subplots_adjust(hspace=0.2)  
plt.show()


In [ ]:

#RMSE

depth = np.arange(160, 1008, 8)

plt.figure(figsize=(10, 8), dpi=300)
plt.subplot(1, 2, 1)
depth_new = np.linspace(depth.min(), depth.max(), 1000)
f1_rmseu = interp1d(depth, u_rm[:,0], kind='cubic')
f1_rmsev = interp1d(depth, v_rm[:,0], kind='cubic')

plt.plot(f1_rmseu(depth_new), depth_new, color='blue', linewidth=2, linestyle='-', label='u')
plt.plot(f1_rmsev(depth_new), depth_new, color='red', linewidth=2, linestyle='-', label='v')

plt.xlabel('RMSE (m/s)', fontsize=15, fontweight='bold')
plt.ylabel('Depth (m)', fontsize=15, fontweight='bold')
plt.xticks([0.005,0.010, 0.015, 0.020], fontsize=13, fontweight='bold') 
plt.yticks(fontsize=13, fontweight='bold')
plt.legend(fontsize=15,loc='lower right')
plt.tick_params(axis='x', direction='in', which='both')  
plt.tick_params(axis='y', direction='in', which='both') 
plt.title('(a)', fontsize=15, fontweight='bold',loc='left',pad=10)

plt.grid()
plt.gca().invert_yaxis()
plt.tight_layout()

plt.subplot(1, 2, 2) 
depth_new = np.linspace(depth.min(), depth.max(), 1000)
f_rmseu = interp1d(depth, rmseu[0,:], kind='cubic')
f_rmsev = interp1d(depth, rmsev[0,:], kind='cubic')
f_rmseuc = interp1d(depth, rmseuc[0,:], kind='cubic')
f_rmsevc = interp1d(depth, rmsevc[0,:], kind='cubic')
plt.plot(f_rmseu(depth_new), depth_new, color='blue', linewidth=2, linestyle='-', label='incoherent u ')
plt.plot(f_rmsev(depth_new), depth_new, color='red', linewidth=2, linestyle='-', label='incoherent v ')
plt.plot(f_rmseuc(depth_new), depth_new, color='blue', linewidth=2, linestyle='--', label='coherent u ')
plt.plot(f_rmsevc(depth_new), depth_new, color='red', linewidth=2, linestyle='--', label='coherent v ')

plt.xlabel('RMSE (m/s)', fontsize=15, fontweight='bold')
plt.ylabel('Depth (m)', fontsize=15, fontweight='bold')
plt.xticks([0.000, 0.005, 0.010, 0.015,0.020], fontsize=12, fontweight='bold') 
plt.yticks(fontsize=12, fontweight='bold')
plt.legend(fontsize=10)
plt.tick_params(axis='x', direction='in', which='both')
plt.tick_params(axis='y', direction='in', which='both') 

plt.title('(b)', fontsize=15, fontweight='bold',loc='left',pad=10)
plt.grid()
plt.gca().invert_yaxis()
plt.tight_layout()

In [ ]:

#point density test by gaussian_kde

from scipy.stats import gaussian_kde
from matplotlib.colors import LogNorm
from sklearn.metrics import r2_score
import matplotlib.ticker as ticker

formatter = ticker.FuncFormatter(lambda x, _: f'{x:>5.2f}')
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = plt.cm.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

truncated_cmap = truncate_colormap(plt.cm.Spectral_r, 0.3, 1.0)

plt.figure(figsize=(16, 20), dpi=300)
x = (u_pre).reshape(-1,)
y = (u).reshape(-1,)
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]
a = plt.scatter(x, y,c=z,  s=10,cmap=truncated_cmap)
x_line = np.linspace(-1, 1, 100)
y_line = x_line
plt.plot(x_line, y_line, color='black',linestyle='--',linewidth=2,alpha=0.7)
plt.xlim(-0.3,0.3)
plt.ylim(-0.3,0.3)
plt.ylabel('Observed u', fontsize=15, fontweight='bold')
plt.xlabel('Forecasted u', fontsize=15, fontweight='bold')
plt.xticks(fontsize=13, fontweight='bold')
plt.yticks(fontsize=13, fontweight='bold')
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2)  
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2)  
cb5 = plt.colorbar(a)
cb5.ax.set_title('PD', fontsize=13, pad=10, fontweight='bold')
cb5.set_ticks([0, 0.2, 0.4, 0.6])
cb5.ax.tick_params(which='major', direction='in', length=5, width=1)
cb5.ax.yaxis.set_tick_params(which='major', labelsize=14, width=1, labelcolor='black')
cb5.ax.yaxis.set_major_formatter(formatter)
plt.grid(True, linestyle='--')
rmse = round(math.sqrt(mean_squared_error(u_pre, u)), 4)
r2 = round(r2_score(u, u_pre), 4)
title_text = f'\n RMSE: {rmse} \n\n R²: {r2}'
plt.text(-0.28, 0.27, title_text, fontsize=16, fontweight='bold', va='top', ha='left')
plt.title('(a) BILTA', fontsize=17, fontweight='bold',loc='left')


In [ ]:

#HKE

plt.figure(figsize=(14, 14), dpi=300)

plt.subplot(4, 1, 1)
plt.plot(time, ke_mode1_obs, label='Mode1', linewidth=0.2,color='#00008B')
plt.plot(time, ke_mode1_obs + ke_mode2_obs, label='Mode2', linewidth=0.2,color='#006400')
plt.plot(time, ke_mode1_obs + ke_mode2_obs + ke_mode3_obs, label='Mode3', linewidth=0.2,color='#8B0000')
plt.plot(time, ke_mode1_obs + ke_mode2_obs + ke_mode3_obs + ke_mode4_obs, label='Mode4', linewidth=0.2,color='#4B0082')
fill1 = plt.fill_between(time, 0, ke_mode1_obs, color='#00008B', alpha=0.3)
fill2 = plt.fill_between(time, ke_mode1_obs, ke_mode1_obs + ke_mode2_obs, color='#006400', alpha=0.3)
fill3 = plt.fill_between(time, ke_mode1_obs + ke_mode2_obs, ke_mode1_obs + ke_mode2_obs + ke_mode3_obs, color='#8B0000', alpha=0.3)
fill4 = plt.fill_between(time, ke_mode1_obs + ke_mode2_obs + ke_mode3_obs, ke_mode1_obs + ke_mode2_obs + ke_mode3_obs + ke_mode4_obs, color='#4B0082', alpha=0.3)
line_all, = plt.plot(time, ke_modeall_obs, linewidth=1, color='k', label='ALL')

plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'], fontsize=14, fontweight='bold')
plt.yticks([0.0,0.5,1.0,1.5,2.0,2.5,3.0],fontsize=14, fontweight='bold')
#plt.xticklabels(['09/10/2022','09/23','10/07','10/21','11/04','11/18','12/02','12/16','12/30','01/13/2023/','01/27','02/10'],fontsize=15)
plt.title('(a) Observed Depth-integrated HKE', fontsize=15, fontweight='bold',loc='left',pad=12)
#ax[1].set_title('(b)The observations ITs Semidiurnal ITs v mode4',fontsize=12)
#ax[2].set_title('(c)140$°$E section difference ',fontsize=12)
#ax[0].invert_yaxis()
#ax[1].invert_yaxis()
#ax[2].invert_yaxis()
plt.xlabel('Time', fontsize=14, fontweight='bold')
plt.ylabel('HKE (kJ/m$^{2}$)', fontsize=14, fontweight='bold')
#ax[1].set_ylabel('Depth (m)')
#ax[2].set_ylabel('Depth (m)')

# 添加图例
plt.legend([fill1, fill2, fill3, fill4, line_all], ['Mode1', 'Mode2', 'Mode3', 'Mode4', 'ALL'], loc='upper right',fontsize=11)
plt.tick_params(axis='x', direction='in', which='both')  
plt.tick_params(axis='y', direction='in', which='both')  


plt.subplot(4, 1, 2)
plt.plot(time, ke_mode1, label='Mode1', linewidth=0.2,color='#00008B')
plt.plot(time, ke_mode1 + ke_mode2, label='Mode2', linewidth=0.2,color='#006400')
plt.plot(time, ke_mode1 + ke_mode2 + ke_mode3, label='Mode3', linewidth=0.2,color='#8B0000')
plt.plot(time, ke_mode1 + ke_mode2 + ke_mode3 + ke_mode4, label='Mode4', linewidth=0.2,color='#4B0082')
fill1 = plt.fill_between(time, 0, ke_mode1, color='#00008B', alpha=0.3)
fill2 = plt.fill_between(time, ke_mode1, ke_mode1 + ke_mode2, color='#006400', alpha=0.3)
fill3 = plt.fill_between(time, ke_mode1 + ke_mode2, ke_mode1 + ke_mode2 + ke_mode3, color='#8B0000', alpha=0.3)
fill4 = plt.fill_between(time, ke_mode1 + ke_mode2 + ke_mode3, ke_mode1 + ke_mode2 + ke_mode3 + ke_mode4, color='#4B0082', alpha=0.3)
line_all, = plt.plot(time, ke_modeall, linewidth=1, color='k', label='ALL')

plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'], fontsize=14, fontweight='bold')
plt.yticks([0.0,0.5,1.0,1.5,2.0,2.5,3.0],fontsize=14, fontweight='bold')
#plt.xticklabels(['09/10/2022','09/23','10/07','10/21','11/04','11/18','12/02','12/16','12/30','01/13/2023/','01/27','02/10'],fontsize=15)
plt.title('(b) Forecasted Depth-integrated HKE', fontsize=15, fontweight='bold',loc='left',pad=12)
#ax[1].set_title('(b)The observations ITs Semidiurnal ITs v mode4',fontsize=12)
#ax[2].set_title('(c)140$°$E section difference ',fontsize=12)
#ax[0].invert_yaxis()
#ax[1].invert_yaxis()
#ax[2].invert_yaxis()
plt.xlabel('Time', fontsize=14, fontweight='bold')
plt.ylabel('HKE (kJ/m$^{2}$)', fontsize=14, fontweight='bold')
#ax[1].set_ylabel('Depth (m)')
#ax[2].set_ylabel('Depth (m)')

plt.legend([fill1, fill2, fill3, fill4, line_all], ['Mode1', 'Mode2', 'Mode3', 'Mode4', 'ALL'], loc='upper right',fontsize=11)
plt.tick_params(axis='x', direction='in', which='both')  
plt.tick_params(axis='y', direction='in', which='both') 



plt.subplot(4, 1, 3)
plt.plot(time, ke_mode1_di, label='Mode1', linewidth=3, linestyle='-', color='#00008B', alpha=0.8)
plt.plot(time, ke_mode2_di, label='Mode2', linewidth=3, linestyle='-', color='#006400', alpha=0.8)
#plt.plot(time, ke_mode3_di, label='Mode3', linewidth=3, linestyle='-', color='#7CFC00')
#plt.plot(time, ke_mode4_di, label='Mode4', linewidth=3, linestyle='-', color='#4682B4')

plt.scatter(time, ke_modeall_di, s=12, color='k', label='ALL')

plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=14, fontweight='bold')
plt.yticks([-1,-0.5,0,0.5,1],fontsize=14, fontweight='bold')
plt.title('(c)', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=14, fontweight='bold')
plt.ylabel('Relative Error', fontsize=14, fontweight='bold')

plt.legend(loc='upper right', fontsize=12, markerscale=2)
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2)  # 对 x 轴刻度线向内
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2)  # 对 y 轴刻度线向内
plt.grid()
plt.tight_layout()


plt.subplot(4, 1, 4)
#plt.plot(time, ke_mode1_di, label='Mode1', linewidth=3, linestyle='-', color='blue')
#plt.plot(time, ke_mode2_di, label='Mode2', linewidth=3, linestyle='-', color='#FF5733')
plt.plot(time, ke_mode3_di, label='Mode3', linewidth=3, linestyle='-', color='#8B0000', alpha=0.8)
plt.plot(time, ke_mode4_di, label='Mode4', linewidth=3, linestyle='-', color='#4B0082', alpha=0.8)


plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=14, fontweight='bold')
plt.yticks([-1,-0.5,0,0.5,1],fontsize=14, fontweight='bold')
plt.title('(d)', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=14, fontweight='bold')
plt.ylabel('Relative Error', fontsize=14, fontweight='bold')

plt.tick_params(axis='x', direction='in', which='both', length=6, width=2)  # 对 x 轴刻度线向内
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2)  # 对 y 轴刻度线向内
plt.legend(loc='upper right', fontsize=12, markerscale=2)
plt.grid()
plt.tight_layout()

In [ ]:

#Time series of coherent and incoherent currents at 300 m

from scipy.interpolate import make_interp_spline
time = np.arange(1320)

idx1=train_size+336

plt.figure(figsize=(14, 14), dpi=300)
plt.subplot(4, 1, 1)

spline = make_interp_spline(time, u_inc)
smooth_time = np.linspace(time.min(), time.max(), 1000) 
smooth_data = spline(smooth_time)
plt.plot(smooth_time, smooth_data, label='Observed', linestyle='-', linewidth=3, alpha=0.8,color='firebrick')


spline1 = make_interp_spline(time, u_inc_pre) 
smooth_time1 = np.linspace(time.min(), time.max(), 1000)  
smooth_data1 = spline1(smooth_time1)
plt.plot(smooth_time1, smooth_data1, label='Forecasted', linestyle='-', linewidth=2, alpha=1,color='dodgerblue')




plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.title('(a) Incoherent u', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=15, fontweight='bold')
plt.ylabel('Velocity (m/s)', fontsize=15, fontweight='bold')

plt.legend(loc='upper right', fontsize=12, markerscale=5)
plt.grid(True, linestyle='--')
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2)  
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2)  


plt.subplot(4, 1, 2)
spline = make_interp_spline(time, v_inc)
smooth_time = np.linspace(time.min(), time.max(), 1000) 
smooth_data = spline(smooth_time)
plt.plot(smooth_time, smooth_data, label='Observed', linestyle='-', linewidth=3, alpha=0.8,color='firebrick')


spline1 = make_interp_spline(time, v_inc_pre) 
smooth_time1 = np.linspace(time.min(), time.max(), 1000)  
smooth_data1 = spline1(smooth_time1)
plt.plot(smooth_time1, smooth_data1, label='Forecasted', linestyle='-', linewidth=2, alpha=1,color='dodgerblue')




plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=12, fontweight='bold')
plt.yticks([-0.10,-0.05,0.00,0.05,0.10],fontsize=12, fontweight='bold')
plt.title('(b) Incoherent v', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=15, fontweight='bold')
plt.ylabel('Velocity (m/s)', fontsize=15, fontweight='bold')

plt.legend(loc='upper right', fontsize=12, markerscale=5)
plt.grid(True, linestyle='--')
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2) 
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2) 


plt.subplot(4, 1, 3)
spline = make_interp_spline(time, u_coh)
smooth_time = np.linspace(time.min(), time.max(), 1000) 
smooth_data = spline(smooth_time)
plt.plot(smooth_time, smooth_data, label='Observed', linestyle='-', linewidth=3, alpha=0.8,color='firebrick')


spline1 = make_interp_spline(time, u_coh_pre) 
smooth_time1 = np.linspace(time.min(), time.max(), 1000)  
smooth_data1 = spline1(smooth_time1)
plt.plot(smooth_time1, smooth_data1, label='Forecasted', linestyle='-', linewidth=2, alpha=1,color='dodgerblue')




plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.title('(c) Coherent u', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=15, fontweight='bold')
plt.ylabel('Velocity (m/s)', fontsize=15, fontweight='bold')

plt.legend(loc='upper right', fontsize=12, markerscale=5)
plt.grid(True, linestyle='--')
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2) 
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2) 


plt.subplot(4, 1, 4)
spline = make_interp_spline(time, v_coh)
smooth_time = np.linspace(time.min(), time.max(), 1000) 
smooth_data = spline(smooth_time)
plt.plot(smooth_time, smooth_data, label='Observed', linestyle='-', linewidth=3, alpha=0.8,color='firebrick')


spline1 = make_interp_spline(time, v_coh_pre) 
smooth_time1 = np.linspace(time.min(), time.max(), 1000)  
smooth_data1 = spline1(smooth_time1)
plt.plot(smooth_time1, smooth_data1, label='Forecasted', linestyle='-', linewidth=2, alpha=1,color='dodgerblue')




plt.xticks([0,167,335,503,671,839, 1007, 1175, 1319],
           ['05/13','05/20','05/27','06/03','06/10','06/17','06/24','07/01','07/07'],
           fontsize=12, fontweight='bold')
plt.yticks([-0.03,-0.02,-0.01,0.00,0.01,0.02,0.03],fontsize=12, fontweight='bold')
plt.title('(d) Coherent v', fontsize=15, fontweight='bold',loc='left',pad=12)
plt.xlabel('Time', fontsize=15, fontweight='bold')
plt.ylabel('Velocity (m/s)', fontsize=15, fontweight='bold')

plt.legend(loc='upper right', fontsize=12, markerscale=5)
plt.grid(True, linestyle='--')
plt.tick_params(axis='x', direction='in', which='both', length=6, width=2) 
plt.tick_params(axis='y', direction='in', which='both', length=6, width=2) 

plt.tight_layout()

plt.show()